In [1]:
import pickle
from datetime import datetime
import pandas as pd 
import os

In [2]:
def load_pickle(data_path):
    with open(data_path,'rb') as f:
        data = pickle.load(f)
    return data
def save_pickle(data,save_path):
    with open(save_path,'wb') as f:
        pickle.dump(data,f)

## 1. convert to Recbole and SFT format

### 1. Recoble format
- category.pickle
- dataset_name.inter
- dataset_name.item

### 2. SFT format
- category.pickle 'category': [item_ids]
- metas.pickle 'item_id': {'item_id', 'title'}
- sequential.pickle 'user_id': [item_ids]

In [5]:
file_path = '/home/wangshuo/codes/rec-scope/constraint_search/data/movies/base/movies_final.pickle'
final_df = load_pickle(file_path)
final_df.head()

,reviewerID,interactions,count
165875,A34YJ2QSN4DLJL,"[(Intrusion, B010CG2ECS, A34YJ2QSN4DLJL, 14611...",17
189241,A3FY9NYZS092CZ,"[(Journey to Paradise, B005OZ4G38, A3FY9NYZS09...",20
127412,A2N6CSN1QBTOIB,[(Charlie Jade - Complete Season 1 Exclusive t...,25
246295,AE0E6UII0VSZA,"[(Halloween 4 VHS, 630130053X, AE0E6UII0VSZA, ...",23
128960,A2NWAHM64KMWV4,"[(Hit &amp; Miss, B008582FCK, A2NWAHM64KMWV4, ...",36


In [6]:
final_df.iloc[0]['interactions']

[('Intrusion',
  'B010CG2ECS',
  'A34YJ2QSN4DLJL',
  1461196800,
  'This movie is not great in the beginning but then it gets better and better.  The girl in it is cute and the guy with the knife is scary.  I felt sorry for him when I found out his motive.  I like the surprise ending.  True it does look like some cheap movie, but it held my attention till the end.',
  ['Movies & TV', 'Genre for Featured Categories', 'Horror']),
 ('The Zero Theorem 2014',
  'B00IRD8JJ6',
  'A34YJ2QSN4DLJL',
  1461628800,
  "The no this no that sign is the most funny.  The great appeal of this movie is that hot nurse and that weird holodeck website experience with her.  I think men would like this movie for the nurse mainly.  The existential angst and the metaphysical stuff is always there for everyone alive to ponder. The kid adds some pep to the movie for sure with his funny language. The first half is dull but colorful and very creative.  The being spied upon thing is what we live in now.  This movie 

In [7]:
import ast
def convert_to_list(tags_str):
    try:
        tags = ast.literal_eval(tags_str)
        all_tags = []
        for tag in tags:
            all_tags.extend(tag)
        return tags
    except:
        return []

In [14]:

sequential = {}
metas ={}
category = {}
iter =[]
item_recbole = []
item_list =[]
COMMON_TAG = 'Movies & TV'
miss_count = 0
miss_titile =0 
empty_count = 0

for index, row in final_df.iterrows():
    user_id = str(row['reviewerID'])
    # interactions(item_title,item_id, timestamp, tags(list))
    interactions = row['interactions']
    item_ids = []

    for interaction in interactions:
        if type(interaction[0]) != str:
            miss_titile += 1
            continue
        item_title = interaction[0].strip()
        item_id = str(interaction[1])
        tags = interaction[5]
        timestamp = interaction[3]
        
        
        if len(tags) == 0:
            tags = [COMMON_TAG]
            miss_count += 1

        for tag in tags:
            tag =tag.strip()
            if tag not in category:
                category[tag] = []
            if item_id not in category[tag]:
                category[tag].append(item_id)
            # category[tag].append(item_id)
        iter.append({"user_id:token":user_id,"item_id:token":item_id,"timestamp:float":float(timestamp)})
        # item_recbole.append({"item_id:token":item_id,"timestamp:float":toTimestamp(timestamp)})
        item_ids.append(item_id)
        metas[item_id]={"title":item_title}
        item_list.append(item_title)
    
    if len(item_ids) <= 3:
        empty_count += 1
        continue
    sequential[user_id]=item_ids
print('miss_count:',miss_count)
print("miss_titile:",miss_titile)
print("empty_count:",empty_count)

miss_count: 9
miss_titile: 0
empty_count: 0


In [15]:
len(metas)

34452

In [17]:
iter_df = pd.DataFrame(iter)
iter_df = iter_df[["user_id:token","item_id:token","timestamp:float"]]
iter_df.to_csv("/home/wangshuo/codes/InstructControllableRec_RLHF/SASRec/dataset/movies/movies.inter",index=False,sep='\t')

In [51]:
save_base_dir = "/home/wangshuo/codes/InstructControllableRec_RLHF/data/dataset/movies/"
save_pickle(sequential,os.path.join(save_base_dir,"sequential.pickle"))
save_pickle(metas,os.path.join(save_base_dir,"metas.pickle"))
save_pickle(category,os.path.join(save_base_dir,"category.pickle"))
save_pickle(item_list,os.path.join(save_base_dir,"items.pickle"))

# data overview

In [49]:
sequential['10158']

['B007U7M0L8', 'B0015ZITNY', '0911121013']

In [48]:
empty_count=0
for key in sequential.keys():
    if len(sequential[key]) <= 2:
        print(key,sequential[key])
        empty_count += 1
print(empty_count)

0


In [4]:
save_base_dir = "/home/wangshuo/codes/InstructControllableRec_RLHF/data/dataset/steam/"
data = load_pickle(os.path.join(save_base_dir,"metas.pickle"))

In [16]:
len(set(category[COMMON_TAG]))

555

In [3]:
data1=load_pickle("/home/wangshuo/codes/scope-rec/snap/0224-steam-method1/Epoch40_SFT_Result_SFTTestSeqRec_Top10.pickle")
data1

[[['Space Rangers HD: A War Apart'],
  ['Imperator: Rome',
   'Mount & Blade II: Bannerlord',
   'Total War: THREE KINGDOMS',
   'X4: Foundations',
   'Total War: WARHAMMER II',
   'RimWorld',
   'Crusader Kings III',
   'Sid Meier’s Civilization® VI',
   'Frostpunk',
   'Total War: WARHAMMER']],
 [['Spellstone'],
  ['Gear Up',
   'AdVenture Capitalist',
   'Clicker Heroes',
   'Gear Up 2',
   'Unturned',
   'Emily is Away',
   'Trove',
   'BLOCKADE 3D',
   'Dead Bits',
   'The Plan']],
 [['Dying Light 2 Stay Human'],
  ['The Last Spell',
   'Trials of Fire',
   'Loop Hero',
   'New World',
   'Age of Empires IV: Anniversary Edition',
   'Wildermyth',
   'Dyson Sphere Program',
   'OUTRIDERS',
   'Tales of Arise',
   'HUMANKIND™',
   'Core Keeper']],
 [['Tower of Time'],
  ['The Elder Scrolls IV: Oblivion® Game of the Year Edition',
   'Cyberpunk 2077',
   'Hades',
   'DOOM Eternal',
   'Resident Evil 3',
   'Sekiro™: Shadows Die Twice - GOTY Edition',
   'Ori and the Will of the Wisps

In [6]:
data2=load_pickle("/home/wangshuo/codes/rec-scope/constraint_search/data/steam/base/items.pickle")

In [7]:
not_in = []
for item in data1:
    if item not in data2:
        not_in.append(item)
print(len(not_in))

0


In [18]:
'The Witcher® 3: Wild Hunt' in data2

True

In [4]:
# 统计一下每个item的平均token个数
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/home/wangshuo/weights/llama2/Llama-2-7b-hf-chat/")

In [15]:
def get_token_num(text):
    token_list = tokenizer.encode(text,add_special_tokens=False)
    return len(tokenizer.encode(text))

In [8]:
def cal_token_num(data_path):
    data = load_pickle(data_path)
    token_num = 0
    for item in data:
        token_num+= get_token_num(item)
    return token_num/len(data)

In [20]:
data_path = "/home/wangshuo/codes/InstructControllableRec_RLHF/data/dataset/movies/items.pickle"
token_num = cal_token_num(data_path)
print(token_num)

9.18444705882353
